In [16]:
# Importing required libraries
import pandas as pd

# Loading the 
info_df = pd.read_csv("/Users/corycates/Documents/GitHub/Super_Hero/Data/superhero_info - superhero_info.csv")
powers_df = pd.read_csv("/Users/corycates/Documents/GitHub/Super_Hero/Data/superhero_powers - superhero_powers.csv")

# Displaying the first few rows of each dataframe for inspection
info_df.head(), powers_df.head()


(                 Hero|Publisher Gender               Race Alignment  \
 0          A-Bomb|Marvel Comics   Male              Human      good   
 1  Abe Sapien|Dark Horse Comics   Male      Icthyo Sapien      good   
 2            Abin Sur|DC Comics   Male            Ungaran      good   
 3     Abomination|Marvel Comics   Male  Human / Radiation       bad   
 4   Absorbing Man|Marvel Comics   Male              Human       bad   
 
   Hair color Eye color Skin color  \
 0    No Hair    yellow    Unknown   
 1    No Hair      blue       blue   
 2    No Hair      blue        red   
 3    No Hair     green    Unknown   
 4    No Hair      blue    Unknown   
 
                                    Measurements  
 0  {'Height': '203.0 cm', 'Weight': '441.0 kg'}  
 1   {'Height': '191.0 cm', 'Weight': '65.0 kg'}  
 2   {'Height': '185.0 cm', 'Weight': '90.0 kg'}  
 3  {'Height': '203.0 cm', 'Weight': '441.0 kg'}  
 4  {'Height': '193.0 cm', 'Weight': '122.0 kg'}  ,
     hero_names              

In [17]:
# Splitting the "Hero|Publisher" column
info_df[['Hero', 'Publisher']] = info_df['Hero|Publisher'].str.split('|', expand=True)
info_df.drop('Hero|Publisher', axis=1, inplace=True)

# Extracting height and weight from the "Measurements" column
info_df['Height'] = info_df['Measurements'].str.extract("'Height': '(\d+.\d+) cm'") 
info_df['Weight'] = info_df['Measurements'].str.extract("'Weight': '(\d+.\d+) kg'")

# Addressing the issue with spaces in numbers (e.g., "100 kg")
info_df['Height'] = info_df['Height'].str.replace(' ', '').astype(float)
info_df['Weight'] = info_df['Weight'].str.replace(' ', '').astype(float)


# Dropping the original "Measurements" column
info_df.drop('Measurements', axis=1, inplace=True)

info_df.head()


,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0,65.0
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185.0,90.0
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Abomination,Marvel Comics,203.0,441.0
4,Male,Human,bad,No Hair,blue,Unknown,Absorbing Man,Marvel Comics,193.0,122.0


In [18]:
# Renaming the "hero_names" column to "Hero"
powers_df.rename(columns={'hero_names': 'Hero'}, inplace=True)

# One-hot-encode the specified powers: Agility, Flight, and Superspeed
for power in ['Agility', 'Flight', 'Superspeed']:
    powers_df[power] = powers_df['Powers'].str.contains(power).astype(int)
    
# Dropping the original 'Powers' column
powers_df.drop('Powers', axis=1, inplace=True)

powers_df.head()


,Hero,Agility,Flight,Superspeed
0,3-D Man,1,0,0
1,A-Bomb,0,0,0
2,Abe Sapien,1,0,0
3,Abin Sur,0,0,0
4,Abomination,0,0,0


In [20]:
# Merging the two dataframes on the "Hero" column
combined_df = pd.merge(info_df, powers_df, left_on='Hero', right_on='Hero', how='inner')

# Dropping the duplicate 'Hero' column from the merged dataframe
combined_df.drop('Hero', axis=1, inplace=True)

combined_df.head()


,Gender,Race,Alignment,Hair color,Eye color,Skin color,Publisher,Height,Weight,Agility,Flight,Superspeed
0,Male,Human,good,No Hair,yellow,Unknown,Marvel Comics,203.0,441.0,0,0,0
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Dark Horse Comics,191.0,65.0,1,0,0
2,Male,Ungaran,good,No Hair,blue,red,DC Comics,185.0,90.0,0,0,0
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Marvel Comics,203.0,441.0,0,0,0
4,Male,Human,bad,No Hair,blue,Unknown,Marvel Comics,193.0,122.0,0,0,0


In [22]:
# Comparing the average weight of superheroes with and without Super Speed
average_weight_with_speed = combined_df[combined_df['Superspeed'] == 1]['Weight'].mean()
average_weight_without_speed = combined_df[combined_df['Superspeed'] == 0]['Weight'].mean()

average_weight_with_speed, average_weight_without_speed


(nan, 113.58963282937366)

In [23]:
# Calculating the average height of heroes for each publisher
average_height_per_publisher = combined_df.groupby('Publisher')['Height'].mean()

average_height_per_publisher


Publisher
DC Comics            181.923913
Dark Horse Comics    176.909091
George Lucas         159.600000
Image Comics         211.000000
Marvel Comics        191.546128
Shueisha             171.500000
Star Trek            181.500000
Team Epic TV         180.750000
Unknown              178.000000
Name: Height, dtype: float64